# 微调 Chinese-BERT-wwm-ext 模型用于三分类情感分析

本 Notebook 用于在魔塔平台上微调 `hfl/chinese-bert-wwm-ext` 模型，
数据源为本地上传的 `评论情感分析结果_三分类_人工纠错.xlsx` 文件，使用 Huggingface `transformers` 库进行训练。

In [1]:
# 安装所需库
!pip install transformers datasets scikit-learn openpyxl

Looking in indexes: https://mirrors.cloud.aliyuncs.com/pypi/simple
DEPRECATION: pytorch-lightning 1.7.7 has a non-standard dependency specifier torch>=1.9.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: omegaconf 2.0.6 has a non-standard dependency specifier PyYAML>=5.1.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of omegaconf or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 23.3.2 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [2]:
# 导入必要的库
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import Dataset
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, f1_score
import torch

2025-05-13 14:27:05.567220: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-13 14:27:05.607916: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-05-13 14:27:06.788014: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:

# 正确读取 CSV 文件
df = pd.read_csv("评论情感分析结果_三分类_人工纠错.csv")
print(df.columns)

# 只保留需要的列
df = df[["comment", "emotion"]].dropna()

# 标签编码
label2id = {label: i for i, label in enumerate(sorted(df["emotion"].unique()))}
id2label = {v: k for k, v in label2id.items()}
df["label"] = df["emotion"].map(label2id)

# 划分训练和验证集
train_df, val_df = train_test_split(df, test_size=0.2, stratify=df["label"], random_state=42)

# 转换为 Huggingface Dataset
train_dataset = Dataset.from_pandas(train_df[["comment", "label"]])
val_dataset = Dataset.from_pandas(val_df[["comment", "label"]])

Index(['name', 'gender', 'level', 'comment', 'likes', 'emotion', 'score',
       'status'],
      dtype='object')


In [5]:
# 定义本地模型路径
model_path = "./modelbert" # 假设你的模型文件在当前目录下的 modelbert 文件夹中

# 加载 tokenizer
tokenizer = BertTokenizer.from_pretrained(model_path)

# 定义 tokenization 函数
def tokenize_function(examples):
    return tokenizer(examples["comment"], padding="max_length", truncation=True, max_length=256) # 你可以根据需要调整 max_length

# 应用 tokenization 到数据集
tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_val_dataset = val_dataset.map(tokenize_function, batched=True)

# 加载模型
# num_labels 需要设置为你的分类类别数量 (这里是 3)
model = BertForSequenceClassification.from_pretrained(model_path, num_labels=len(label2id), id2label=id2label, label2id=label2id)

# 定义评估指标
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = predictions.argmax(axis=1)
    accuracy = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions, average='weighted') # 对于多分类使用 weighted f1
    return {"accuracy": accuracy, "f1": f1}

# 定义训练参数
training_args = TrainingArguments(
    output_dir="./results",             # 训练输出目录
    num_train_epochs=10,                 # 训练的总 epoch 数，可以根据需要调整
    per_device_train_batch_size=64,     # 每个设备的训练 batch size
    per_device_eval_batch_size=64,      # 每个设备的评估 batch size
    warmup_steps=500,                   # warmup 步数
    learning_rate=3e-5,                 # 学习率
    weight_decay=0.01,                  # weight decay
    logging_dir="./logs",               # log 目录
    logging_steps=10,                   # 每隔多少步记录一次 log
    evaluation_strategy="epoch",        # 每个 epoch 结束后进行评估
    save_strategy="epoch",              # 每个 epoch 结束后保存模型
    load_best_model_at_end=True,        # 训练结束后加载在验证集上表现最好的模型
    metric_for_best_model="f1",         # 用 f1 作为选择最好模型的指标
    greater_is_better=True,             # f1 越大越好
    report_to="none",                   # 不上传到任何平台，如果需要可以改为 "tensorboard" 或其他
)

# 初始化 Trainer
trainer = Trainer(
    model=model,                         # 要训练的模型
    args=training_args,                  # 训练参数
    train_dataset=tokenized_train_dataset, # 训练数据集
    eval_dataset=tokenized_val_dataset,  # 评估数据集
    compute_metrics=compute_metrics,     # 评估指标函数
)

# 开始训练
print("开始训练...")
trainer.train()

# 评估训练完成后的模型（加载的是最好的模型）
print("训练完成，正在评估最终模型...")
eval_results = trainer.evaluate()
print(f"最终评估结果: {eval_results}")

# 你现在可以保存最终的模型和 tokenizer
trainer.save_model("./final_model")
tokenizer.save_pretrained("./final_model")

Map:   0%|          | 0/2326 [00:00<?, ? examples/s]

Map:   0%|          | 0/582 [00:00<?, ? examples/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ./modelbert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Detected kernel version 4.19.91, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


开始训练...


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.840400,0.776739,0.621993,0.491917
2,0.713500,0.670688,0.625430,0.484861
3,0.610800,0.573873,0.731959,0.726990
4,0.500900,0.474335,0.788660,0.789913
5,0.369200,0.448646,0.798969,0.792224
6,0.264900,0.542722,0.807560,0.795235
7,0.210400,0.500573,0.807560,0.802721
8,0.156800,0.623886,0.780069,0.774986
9,0.063500,0.925155,0.774914,0.767471
10,0.099500,0.838126,0.774914,0.767471


训练完成，正在评估最终模型...


最终评估结果: {'eval_loss': 0.5005733370780945, 'eval_accuracy': 0.8075601374570447, 'eval_f1': 0.8027207865194665, 'eval_runtime': 2.6099, 'eval_samples_per_second': 222.996, 'eval_steps_per_second': 3.832, 'epoch': 10.0}


('./final_model/tokenizer_config.json',
 './final_model/special_tokens_map.json',
 './final_model/vocab.txt',
 './final_model/added_tokens.json')